# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [2]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [3]:
!python create_tables.py

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files("data/song_data")

In [6]:
filepath = song_files[0]

In [7]:
songdf = pd.read_json(filepath, lines=True)
songdf.head()

artist_id  artist_latitude artist_location  artist_longitude  \
0  ARNNKDK1187B98BBD5         45.80726  Zagreb Croatia           15.9676   

  artist_name   duration  num_songs             song_id  \
0        Jinx  407.37914          1  SOFNOQK12AB01840FC   

                         title  year  
0  Kutt Free (DJ Volume Remix)     0

In [8]:
songdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 10 columns):
artist_id           1 non-null object
artist_latitude     1 non-null float64
artist_location     1 non-null object
artist_longitude    1 non-null float64
artist_name         1 non-null object
duration            1 non-null float64
num_songs           1 non-null int64
song_id             1 non-null object
title               1 non-null object
year                1 non-null int64
dtypes: float64(3), int64(2), object(5)
memory usage: 160.0+ bytes


## #1: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = songdf[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0].tolist()
artist_data

['ARNNKDK1187B98BBD5', 'Jinx', 'Zagreb Croatia', 45.80726, 15.967600000000001]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [11]:
song_data = songdf[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0].tolist()
song_data

['SOFNOQK12AB01840FC',
 'Kutt Free (DJ Volume Remix)',
 'ARNNKDK1187B98BBD5',
 0,
 407.37914]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [12]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [13]:
log_files = get_files('data/log_data')

In [14]:
filepath = log_files[0]

In [15]:
logdf = pd.read_json(filepath, lines=True)
logdf.head()

artist       auth firstName gender  itemInSession  lastName  \
0  Great Lake Swimmers  Logged In     Kevin      M              0  Arellano   
1  Soziedad Alkoholika  Logged In     Kevin      M              1  Arellano   
2      Franz Ferdinand  Logged In     Kevin      M              2  Arellano   
3         Modest Mouse  Logged In     Kevin      M              3  Arellano   
4         Adam Lambert  Logged In     Kevin      M              4  Arellano   

      length level                 location method      page  registration  \
0  215.11791  free  Harrisburg-Carlisle, PA    PUT  NextSong  1.540007e+12   
1  204.74730  free  Harrisburg-Carlisle, PA    PUT  NextSong  1.540007e+12   
2  172.01587  free  Harrisburg-Carlisle, PA    PUT  NextSong  1.540007e+12   
3  209.52771  free  Harrisburg-Carlisle, PA    PUT  NextSong  1.540007e+12   
4  266.44853  free  Harrisburg-Carlisle, PA    PUT  NextSong  1.540007e+12   

   sessionId                       song  status             ts  \
0        815           Your Rocky Spine     200  1542931645796   
1        815                    Va Bien     200  1542931860796   
2        815  Eleanor Put Your Boots On     200  1542932064796   
3        815                   Float On     200  1542932236796   
4        815                  Aftermath     200  1542932445796   

                                           userAgent userId  
0  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     66  
1  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     66  
2  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     66  
3  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     66  
4  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     66

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [16]:
logdf = logdf.query('page == "NextSong"').copy()
logdf.head()

artist       auth firstName gender  itemInSession  lastName  \
0  Great Lake Swimmers  Logged In     Kevin      M              0  Arellano   
1  Soziedad Alkoholika  Logged In     Kevin      M              1  Arellano   
2      Franz Ferdinand  Logged In     Kevin      M              2  Arellano   
3         Modest Mouse  Logged In     Kevin      M              3  Arellano   
4         Adam Lambert  Logged In     Kevin      M              4  Arellano   

      length level                 location method      page  registration  \
0  215.11791  free  Harrisburg-Carlisle, PA    PUT  NextSong  1.540007e+12   
1  204.74730  free  Harrisburg-Carlisle, PA    PUT  NextSong  1.540007e+12   
2  172.01587  free  Harrisburg-Carlisle, PA    PUT  NextSong  1.540007e+12   
3  209.52771  free  Harrisburg-Carlisle, PA    PUT  NextSong  1.540007e+12   
4  266.44853  free  Harrisburg-Carlisle, PA    PUT  NextSong  1.540007e+12   

   sessionId                       song  status             ts  \
0        815           Your Rocky Spine     200  1542931645796   
1        815                    Va Bien     200  1542931860796   
2        815  Eleanor Put Your Boots On     200  1542932064796   
3        815                   Float On     200  1542932236796   
4        815                  Aftermath     200  1542932445796   

                                           userAgent userId  
0  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     66  
1  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     66  
2  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     66  
3  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     66  
4  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     66

In [17]:
logdf['timestamp'] = pd.to_datetime(logdf.ts, unit ='ms')
logdf.head()

artist       auth firstName gender  itemInSession  lastName  \
0  Great Lake Swimmers  Logged In     Kevin      M              0  Arellano   
1  Soziedad Alkoholika  Logged In     Kevin      M              1  Arellano   
2      Franz Ferdinand  Logged In     Kevin      M              2  Arellano   
3         Modest Mouse  Logged In     Kevin      M              3  Arellano   
4         Adam Lambert  Logged In     Kevin      M              4  Arellano   

      length level                 location method      page  registration  \
0  215.11791  free  Harrisburg-Carlisle, PA    PUT  NextSong  1.540007e+12   
1  204.74730  free  Harrisburg-Carlisle, PA    PUT  NextSong  1.540007e+12   
2  172.01587  free  Harrisburg-Carlisle, PA    PUT  NextSong  1.540007e+12   
3  209.52771  free  Harrisburg-Carlisle, PA    PUT  NextSong  1.540007e+12   
4  266.44853  free  Harrisburg-Carlisle, PA    PUT  NextSong  1.540007e+12   

   sessionId                       song  status             ts  \
0        815           Your Rocky Spine     200  1542931645796   
1        815                    Va Bien     200  1542931860796   
2        815  Eleanor Put Your Boots On     200  1542932064796   
3        815                   Float On     200  1542932236796   
4        815                  Aftermath     200  1542932445796   

                                           userAgent userId  \
0  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     66   
1  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     66   
2  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     66   
3  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     66   
4  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     66   

                timestamp  
0 2018-11-23 00:07:25.796  
1 2018-11-23 00:11:00.796  
2 2018-11-23 00:14:24.796  
3 2018-11-23 00:17:16.796  
4 2018-11-23 00:20:45.796

In [18]:
t = pd.to_datetime(logdf.ts, unit ='ms')
t.head()

0   2018-11-23 00:07:25.796
1   2018-11-23 00:11:00.796
2   2018-11-23 00:14:24.796
3   2018-11-23 00:17:16.796
4   2018-11-23 00:20:45.796
Name: ts, dtype: datetime64[ns]

In [19]:
time_data = (t, t.dt.hour, t.dt.day, t.dt.weekofyear, t.dt.month, t.dt.year, t.dt.dayofweek)
column_labels = ("timestamp", "hour", "day", "week_of_year", "month", "year", "weekday")

In [20]:
time_df = pd.DataFrame.from_dict(dict(zip(column_labels, time_data)))
time_df.head()

timestamp  hour  day  week_of_year  month  year  weekday
0 2018-11-23 00:07:25.796     0   23            47     11  2018        4
1 2018-11-23 00:11:00.796     0   23            47     11  2018        4
2 2018-11-23 00:14:24.796     0   23            47     11  2018        4
3 2018-11-23 00:17:16.796     0   23            47     11  2018        4
4 2018-11-23 00:20:45.796     0   23            47     11  2018        4

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [21]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [22]:
user_df = logdf[['userId', 'firstName', 'lastName', 'gender', 'level']].drop_duplicates()

user_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26 entries, 0 to 298
Data columns (total 5 columns):
userId       26 non-null object
firstName    26 non-null object
lastName     26 non-null object
gender       26 non-null object
level        26 non-null object
dtypes: object(5)
memory usage: 1.2+ KB


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [23]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [24]:
for index, row in logdf.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = [row.timestamp, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent]
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [25]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

In [4]:
!python etl.py

71 files found in data/song_data
1/71 files processed.
2/71 files processed.
3/71 files processed.
4/71 files processed.
5/71 files processed.
6/71 files processed.
7/71 files processed.
8/71 files processed.
9/71 files processed.
10/71 files processed.
11/71 files processed.
12/71 files processed.
13/71 files processed.
14/71 files processed.
15/71 files processed.
16/71 files processed.
17/71 files processed.
18/71 files processed.
19/71 files processed.
20/71 files processed.
21/71 files processed.
22/71 files processed.
23/71 files processed.
24/71 files processed.
25/71 files processed.
26/71 files processed.
27/71 files processed.
28/71 files processed.
29/71 files processed.
30/71 files processed.
31/71 files processed.
32/71 files processed.
33/71 files processed.
34/71 files processed.
35/71 files processed.
36/71 files processed.
37/71 files processed.
38/71 files processed.
39/71 files processed.
40/71 files processed.
41/71 files processed.
42/71 files processed.
43/71 file